TODO :

definir un modèle probabiliste "conditionnel" hiérarchique

et le tester en premier avec info complete (tous les pixels)

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
from waveimage import WaveImage

In [ ]:
import math

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

#### Ajout des bords

In [ ]:
def mnist_reshape_32(x):
    assert x.shape == (28 * 28,)
    image = x.reshape(28,28)
    image = np.append(np.zeros((2,28)), image, axis = 0)
    image = np.append(image, np.zeros((2,28)), axis = 0)
    image = np.append(np.zeros((32,2)), image, axis = 1)
    image = np.append(image, np.zeros((32,2)), axis = 1)
    return image

image = mnist_reshape_32(mnist.train.images[0])


#### insertion de quatre points (avec leur descendance)

In [ ]:
#image = image[:3,:]
w1 = WaveImage()
w2 = WaveImage(image = image)
w1.add_coeffs(w2, u = (15, 15), h_ref = w2.get_h_max())
w1.add_coeffs(w2, u = (15, 17), h_ref = w2.get_h_max())
w1.add_coeffs(w2, u = (17, 15), h_ref = w2.get_h_max())
w1.add_coeffs(w2, u = (17, 17), h_ref = w2.get_h_max())


In [ ]:
plt.figure(figsize = (20,20))
plt.subplot(1,2,1)
plt.imshow(w2.get_image(), cmap = 'gray_r', interpolation = 'nearest')
plt.plot([-0.5, 31.5],[15.5, 15.5],'r--')
plt.plot([15.5, 15.5],[-0.5, 31.5],'r--')
plt.xlim([-0.5, 31.5])
plt.ylim([-0.5, 31.5])
plt.gca().invert_yaxis()
plt.subplot(1,2,2)
plt.imshow(w1.get_image(), cmap = 'gray_r', interpolation = 'nearest')
plt.plot([-0.5, 31.5],[15.5, 15.5],'r--')
plt.plot([15.5, 15.5],[-0.5, 31.5],'r--')
plt.plot(15 - .5, 15 - .5, 'r+', markersize = 12)
plt.plot(15 - .5, 17 - .5, 'r+', markersize = 12)
plt.plot(17 - .5, 15 - .5, 'r+', markersize = 12)
plt.plot(17 - .5, 17 - .5, 'r+', markersize = 12)
plt.xlim([-0.5, 31.5])
plt.ylim([-0.5, 31.5])
plt.gca().invert_yaxis()

#### Test de la fonction de copie

In [ ]:
w3 = w2.copy()

h_max = w3.get_h_max()
data =  w3.get_data()
h_ref = h_max - 1
for u in data[h_ref]:
    w3.set_data(h_ref, u, np.zeros(3))

plt.subplot(1,2,1)
plt.imshow(w3.get_image(), cmap = 'gray_r', interpolation = 'nearest')
plt.plot([-0.5, 31.5],[15.5, 15.5],'r--')
plt.plot([15.5, 15.5],[-0.5, 31.5],'r--')
plt.xlim([-0.5, 31.5])
plt.ylim([-0.5, 31.5])
plt.gca().invert_yaxis()

w3 = w2.copy()
data =  w3.get_data()

for u in data[0]:
    w3.set_data(0, u, 0)
for h in range(1,h_ref):
    for u in data[h]: 
        w3.set_data(h, u, np.zeros(3))
        
plt.subplot(1,2,2)
plt.imshow(w3.get_image(), cmap = 'gray_r', interpolation = 'nearest')
plt.plot([-0.5, 31.5],[15.5, 15.5],'r--')
plt.plot([15.5, 15.5],[-0.5, 31.5],'r--')
plt.xlim([-0.5, 31.5])
plt.ylim([-0.5, 31.5])
plt.gca().invert_yaxis()


#### Base d'apprentissage

In [ ]:
B_app = []
for i in range(len(mnist.train.images)):
    if i % 1000 == 0 :
        print i
    c = mnist.train.labels[i]
    image =  mnist_reshape_32(mnist.train.images[i])
    B_app += [(WaveImage(image = image), c)]
    

#### Base de test

In [ ]:
B_test = []
for i in range(len(mnist.test.images)):
    if i % 1000 == 0 :
        print i
    c = mnist.test.labels[i]
    image =  mnist_reshape_32(mnist.test.images[i])
    B_test += [(WaveImage(image = image), c)]
    